# Optimización del diseño de una viga por Estrategias Evolutivas

## Introducción al problema

Los elementos estructurales como vigas o columnas se deben de diseñar por medio de un proceso iterativo de prueba y error. Debido a que en este proceso se deben de revisar múltiples restricciones, en muchas ocasiones los proyectistas conservan la primera solución que cumple con todas las condiciones impuestas por la normativa correspondiente. Al emplear la primera solución factible, el costo de ésta dependerá en gran medida de la pericia y experiencia del proyectista que diseña el elemento estructural, resultando evidente que aquellas y aquellos con menor práctica presentarán diseños con los costos de construcción más elevados.

El encontrar el diseño óptimo de una viga de concreto reforzado es un problema de optimización combinatoria que ha sido planteado anteriormente por Coello et al.[1]. De manera similar a lo realizado por él, aquí se afrontará el problema por medio de el algoritmo conocido como Estrategias Evolutivas, optimizándose el diseño de una viga biapoyada con una luz de 12 metros sometida a una carga viva uniformemente distribuida W. En la figura se muestra en esquema de la estructura considerada en este ejemplo. Como normativa de referencia se utilizan las Normas Técnicas Complementarias (NTC) del Reglamento de Construcción de la Ciudad de México[2]. 


<img src= "https://raw.githubusercontent.com/LuisVelasc/Imagenes/main/OptimizacionVigaEE1.PNG" alt="drawing" width="400">

Las Estrategias Evolutivas fueron creadas por Rechenberg[3] en la década de los 60’s, en la Universidad Técnica de Berlín, y posteriormente desarrolladas por Schwefel[4] en la década de los 70´s. Al igual que Algoritmos Genéticos, esta metaheurística aplica los principios de la selección natural en la optimización de problemas, sin embargo, los operadores utilizados difieren ligeramente entre ambas metaheurísticas. 

## Variables del código

Se defienen los factores que controlan el desempeño del algoritmo, estos son: el tamaño de la población con la que trabajará Estategias Evolutivas, el número de generaciones que creará el algoritmo y la probabilidad de mutación de las nuevas soluciones creadas. 

In [1]:
Mu=10;         % Tamaño de la población
GenMax=40000; % Número máximo de generaciones
PM=0.50;      % Probabilidad mutacion hacia la vecindad

Intervalo de valores para el cual existen nuestras variables de decisión. Para facilitar su lectura, se muestra un esquema donde se indica la ubicación de cada variable en el elemento.

<img src= "https://raw.githubusercontent.com/LuisVelasc/Imagenes/main/OptimizacionVigaEE2.PNG" alt="drawing" width="500">

In [2]:
Vfc=(25:5:70);             % fc = Resistencia del concreto (Probar con f'c>40 unicamente)
VH=(0.30:0.05:4.0);        % H = Peralte de la sección
VB=(0.25:0.05:3.0);        % B = Ancho de la sección
Vn=(2:1:30);               % nc,n1,n2 = N barras (compresión, tensión sup, tensión inf)
Vfi=[2.5,3,4,5,6,8,10,12]; % fic,fi1,fi2 = Diámetro acero (compresión, tensión sup, tensión inf)
VnE=[1,2];                 % nE = N de estribos
VfiE=[2.5,3];              % fiE = Diámetro de los estribos
VsE=(0.075:0.025:0.30);    % sE = Separación de los estribos (Apoyos y centro)

Parámetros del problema, son variables que se requieren para definir nuestro problema pero que se mantienen inalterados durante el proceso de optimización.

In [3]:
L=12;         % Longitud viga, m
BApoyo=0.30;  % Ancho de los apoyos, m
rec=0.04;     % Recubrimiento, m
w=2.5;        % Carga distribuida servicio, kN/m
Wm=6.25;      % Carga distribuida rotura, kN/m

Se crean matrices que almacenarán los resultados intermedios del algoritmo.

In [4]:
Poblacion=zeros(Mu,15);
Vv=zeros(13,max([size(Vfc,2),size(VH,2),size(VB,2),size(Vn,2),size(VsE,2)]));
Costos=zeros(Mu,1);
Solucion=zeros(1,15); 
Cmin=zeros(GenMax,1);

## Tamaño del espacio de soluciones

Se determina el tamaño del espacio de soluciones en función de la cardinalidad de las variables de decisión de nuestro problema. 

In [5]:
EspacioSoluciones = size(Vfc,2)*size(VH,2)*size(VB,2)*size(Vn,2)^3*...
    size(Vfi,2)^3*size(VnE,2)*size(VfiE,2)*size(VsE,2);
    
fprintf('\n')
fprintf('%s posibles diseños de vigas.\n', EspacioSoluciones)
fprintf('')


2.097844e+13 posibles diseños de vigas.



## Funciones del algoritmo

La siguiente función es el bloque de comprobaciones de nuestro algoritmo. Determina si una viga propuesta es una solución factible. Las comprobaciones se basan en los especificado por las Normas Técnicas Complementarias para el Reglamento de Construcción de la Ciudad de México. 

In [6]:
%%file ComprobacionesAG.m

function [Check,Le,B,H,fc,d,Asc,As1,As2,Av,sEApoyo,sECentro] = ComprobacionesAG(Viga,L,rec,BApoyo,Wm,w)
% Esta función determina la validez de la viga y define la longitud 
% donde se colocarán los estribos en el apoyo

% Se llaman los valores de la viga
fc=Viga(1,2);
H=Viga(1,3);
B=Viga(1,4);
nc=Viga(1,5);
fic=Viga(1,6)*(2.54/800);
ns1=Viga(1,7);
fis1=Viga(1,8)*(2.54/800);
ns2=Viga(1,9);
fis2=Viga(1,10)*(2.54/800);
nE=Viga(1,11);
fiE=Viga(1,12)*(2.54/800);
sEApoyo=Viga(1,13);
sECentro=Viga(1,14);

% Calculos de apoyo
fcc=0.85*fc;    % Es f''c, MPa
if(fc>28)       % Altura efectiva bloque de compresión, m
    beta1=1.05-fc/140;
else
    beta1=0.85;
end
if(fc>=40)      % Módulo elasticidad concreto, MPa
    Ec=2700*fc^(1/2)+11000;
else
    Ec=4400*fc^(1/2);
end

fy=420;     % Esfuerzo de fluencia del acero, MPa
Es=200000;  % Módulo de elásticidad del acero, MPa

Nu=Es/Ec;   % Relación entre módulos

Asc=nc*pi()/4*(fic)^2;      % Área a compresión, m2
As1=ns1*pi()/4*(fis1)^2;    % Área a tensión superior, m2
As2=ns2*pi()/4*(fis2)^2;    % Área a tensión inferior, m2
Av=2*nE*pi()/4*(fiE)^2;     % Área estribos apoyo, m2

ro=(As1+As2)/(B*H); % Cuantía a tensión
roc=Asc/(B*H); % Cuantía a compresión


sepV = 0.04; % Separación entre armaduras verticales, m
BApoyo = 0.30; % Ancho de los apoyos, m

Yg=(As1*(sepV+fis1/2+fis2/2))/(As1+As2); % Centro de gravedad de las armaduras, m
d=H-rec-fiE-As2/2-Yg; % Es d (peralte efectivo), m
dc=rec+fiE+fic/2; % Es d', m
Le=0; 
Check=1;

% Relación geométrica
if(H/B>6)
    Check=1;
    return
end

% Separación entre refuerzos longitudinales
if((B-2*rec-2*fiE-nc*fic)/(nc-1)<0.04)
    Check=2;
    return
end

if((B-2*rec-2*fiE-ns1*fis1)/(ns1-1)<0.04)
    Check=3;
    return
end

if((B-2*rec-2*fiE-ns2*fis2)/(ns2-1)<0.04)
    Check=4;
    return
end

% Revisión flexión
a=(As1+As2-Asc)*fy/(fcc*B); % Profundidad bloque de compresión
Mflex=1.5*Wm*L^2/8+1.3*(25*H*B)*L^2/8;

% Acero a tensión mínimo
if(As1+As2<(0.22*(fc*10^3)^(1/2)*B*d/(fy*10^3)))
    Check=5;
    return
end

% Acero máximo a tensión
if(As1+As2>0.90*(600*beta1/(600+fy)*fcc*B*d/(fy)+Asc))
    Check=6;
    return
end

% Fluencia del acero a compresión
if(As1+As2-Asc<600*beta1*fcc*B*dc/((600-fy)*fy))
    Check=7;
    return
end

% Momento de rotura
if(Mflex>0.90*((As1+As2-Asc)*fy*(d-a/2)+Asc*fy*(d-dc))*10^3)
    Check=8;
    return
end

% Revisión cortante
Vu=1.5*Wm*L/2-1.5*Wm*(BApoyo/2+d)+1.3*(25*B*H)*L/2-1.3*(25*B*H)*(BApoyo/2+d);
if(ro<0.015)
    Vcr=0.75*(0.2+20*ro)*0.3*(fc*10^3)^(1/2)*B*d;
else
    Vcr=0.75*0.16*(fc*10^3)^(1/2)*B*d;
end
VsR=0.75*Av*fy*(10^3)*d/sEApoyo;

% Vcr máximo
if(Vcr>0.75*(0.47)*(fc*10^3)^(1/2)*B*d)
    Check=9;
    return
end

% Vu máximo
if(Vu>0.75*(0.80)*(fc*10^3)^(1/2)*B*d)
    Check=10;
    return
end

% Separaciones máximas estribos
if(Vu>0.75*(0.47)*(fc*10^3)^(1/2)*B*d)
    if(sEApoyo>0.25*d)
        Check=11;
        return
    end
else
    if(Vu>Vcr)
        if(sEApoyo>0.50*d)
            Check=12;
            return
        end
    end
end

% Resistencia a cortante
if(Vcr+VsR<Vu)
    Check=13;
    return
end

% Se define cambio de separación de estribos
VsRc=0.75*Av*fy*(10^3)*d/sECentro/nE;
Le=(1.5*Wm*L/2+1.3*25*B*H*L/2-Vcr-VsRc)/(1.5*Wm+1.3*25*B*H);

% Comprobaciones de deflexión
AFN=B/2;
BFN=Asc*(Nu-1)+(As1+As2)*Nu;
CFN=-Asc*(Nu-1)*dc-(As1+As2)*Nu*d;
FN=(-BFN+(BFN^2-4*AFN*CFN)^(1/2))/(2*AFN);

Iag=B*FN^3/12+B*FN^3/4+(Nu-1)*Asc*(FN-dc)^2+Nu*(As1+As2)*(FN-d)^2;

% Deflexión elástica
Delas=5*w*L^4/(384*Ec*10^3*Iag);

% Deflexión diferida
Ddifer=2/(1+50*roc)*Delas;

% Deflexión total
if(Delas+Ddifer>L/240)
    Check=14;
    return
end

% Comprobaciones de agrietamiento
fsMax=40000;
fsAcero=(w*L^2/8)*(d-FN)*Nu/Iag;
df=rec+fiE+fis2/2;
Afis=B*(Yg+df)*2/((As1+As2)/min(As1/ns1,As2/ns2));
h1=d-FN;
h2=H-FN;

if(fsMax<fsAcero*(df*Afis)^(1/3)*h2/h1)
    Check=15;
    return
end

Check=0;

% Fin de la función
end

Created file 'C:\Users\fable\Documents\Jupyter Notebook\OptimizacionVigaEE\ComprobacionesAG.m'.


Función que determina el coste de las vigas. Para estimar el coste de las soluciones solo se toman en cuenta los precios unitarios de la cimbra, el acero y el concreto. Los precios son estimados y no se han determinado por algún analisis de costos. 

In [7]:
%%file CostoViga.m

function [Costo]=CostoViga(L,Le,rec,BApoyo,B,H,fc,d,Asc,As1,As2,Av,sEApoyo,sECentro,Vfc)
% Esta función determina el costo de la viga actual

% Costos de los materiales
Cfc = [1103.2, 1253.7, 1354.8, 1420.3, 1502.5, 1570.1, 1668.2, 1722.9, 1777.6, 1832.3]; % MXN/m3
Cacero = 19.64; % MXN/kg
Ccimbra = 220.0; % MXN/m2

% Costo del concreto
Concreto=(L+BApoyo)*B*H*Cfc(1,find(Vfc==fc));

% Costo del acero longitudinal
Longitudinal=(L+BApoyo+H)*(Asc+As1+As2)*Cacero*7860;

% Costo estribos
if(Le-BApoyo/2-d<0)
    Lc=L-2*(sEApoyo+BApoyo/2+d);
    EstribosApoyo=2*(1+1)*(B-2*rec+H-2*rec)*Av*Cacero*7860;
    EstribosCentro=(round(Lc/sECentro)-1)*(B-2*rec+H-2*rec)*Av*Cacero*7860;
else
    Lc=L-(2*round((Le-BApoyo/2-d)/sEApoyo)*sEApoyo+d+BApoyo/2);
    EstribosApoyo=2*(round((Le-BApoyo/2-d)/sEApoyo)+1)*(B-2*rec+H-2*rec)*Av*Cacero*7860;
    EstribosCentro=(round(Lc/sECentro)-1)*(B-2*rec+H-2*rec)*Av*Cacero*7860;
end

% Costo cimbra
Cimbra=(L+BApoyo)*(B+2*H)*Ccimbra;

% Costo total
Costo=Concreto+Longitudinal+EstribosApoyo+EstribosCentro+Cimbra;
end

Created file 'C:\Users\fable\Documents\Jupyter Notebook\OptimizacionVigaEE\CostoViga.m'.


Función que muta el valor de una variable de decisión hacia una posición en la vecindad inmediata del valor actual. Es un movimiento de mutación de corto alcance. 

In [8]:
%%file EEMutacion.m

function [Mut] = EEMutacion(Solucion,Vv,i)
% Función que define la variable a mutar y cómo hacerlo
Xx=Vv(i,1:find(Vv(i,:)==0,1)-1); % Valores de la variable

if(size(Xx,2)==0)
    Xx=Vv(i,:);
end

p=find(Solucion(1,i+1)==Xx);    % Posición en el vector
m=randi(2);                     % Dirección del cambio

if(m==1)
    if(p~=1)
        Mut=Xx(1,p-1);  % Nuevo valor
    else
        Mut=Xx(1,p);    % No se altera el valor
    end
else
    if(p~=size(Xx,2))
        Mut=Xx(1,p+1);  % Nuevo valor
    else
        Mut=Xx(1,p);    % No se altera el valor
    end
end
end

Created file 'C:\Users\fable\Documents\Jupyter Notebook\OptimizacionVigaEE\EEMutacion.m'.


Función que muta el valor de una variable de decisión hacia un valor aleatorio en el alfabeto de la variable de decisión. Este operador permite que el valor actual mute hacia cualquier otro valor existente en el intervalo de la variable de decisión.

In [9]:
%%file EEMutacion2.m
function [Mut] = EEMutacion2(Vv,i)
% Función que define la variable a mutar y cómo hacerlo
Xx=Vv(i,:);     % Valores de la variable

if(size(find(Xx==0,1),2)==0)
    Mut=Xx(1,randi(size(Xx,2)));   % Mutación aleatoria
else
    Mut=Xx(1,randi(find(Xx==0,1)-1));   % Mutación aleatoria
end

Created file 'C:\Users\fable\Documents\Jupyter Notebook\OptimizacionVigaEE\EEMutacion2.m'.


## Algoritmo de Estrategias Evolutivas

Se comienza agrupando las variables de decisión en una matriz. Este arreglo facilitará trabajar con los operadores mutación. 

In [10]:
Vv(1,1:size(Vfc,2))=Vfc;
Vv(2,1:size(VH,2))=VH;
Vv(3,1:size(VB,2))=VB;
Vv(4,1:size(Vn,2))=Vn;
Vv(5,1:size(Vfi,2))=Vfi;
Vv(6,1:size(Vn,2))=Vn;
Vv(7,1:size(Vfi,2))=Vfi;
Vv(8,1:size(Vn,2))=Vn;
Vv(9,1:size(Vfi,2))=Vfi;
Vv(10,1:size(VnE,2))=VnE;
Vv(11,1:size(VfiE,2))=VfiE;
Vv(12,1:size(VsE,2))=VsE;
Vv(13,1:size(VsE,2))=VsE;

Se genera la población inicial del algoritmo de manera aleatoria. Los valores de las variables de decisión que componen a esta población inicial son escogidos siguiendo una distribución de probabilidad uniforme.

In [11]:
t=clock;
rng(t(1,6))
i=1; % Para iniciar el while
while i<=Mu
    Poblacion(i,1)=0;
    Poblacion(i,2)=Vv(1,randi(size(Vfc,2)));
    Poblacion(i,3)=Vv(2,randi(size(VH,2)));
    Poblacion(i,4)=Vv(3,randi(size(VB,2)));
    Poblacion(i,5)=Vv(4,randi(size(Vn,2)));
    Poblacion(i,6)=Vv(5,randi(size(Vfi,2)));
    Poblacion(i,7)=Vv(4,randi(size(Vn,2)));
    Poblacion(i,8)=Vv(5,randi(size(Vfi,2)));
    Poblacion(i,9)=Vv(4,randi(size(Vn,2)));
    Poblacion(i,10)=Vv(5,randi(size(Vfi,2)));
    Poblacion(i,11)=Vv(10,randi(size(VnE,2)));
    Poblacion(i,12)=Vv(11,randi(size(VfiE,2)));
    Poblacion(i,13)=Vv(12,randi(size(VsE,2)));
    Poblacion(i,14)=Vv(12,randi(size(VsE,2)));
    
    % Comprobaciones de la solución
    [Check,Le,B,H,fc,d,Asc,As1,As2,Av,sEApoyo,sECentro]=...
        ComprobacionesAG(Poblacion(i,1:14),L,rec,BApoyo,Wm,w);
    Poblacion(i,1)=Check;
    Poblacion(i,15)=Le;
  
    if(Poblacion(i,1)==0)
        [Costo]=CostoViga(L,Le,rec,BApoyo,B,H,fc,d,Asc,As1,As2,Av,sEApoyo,sECentro,Vfc);
        Costos(i,1)=Costo;
        i=i+1;
    end
end

Se determina e imprime el costo de la solución de mayor calidad en nuestra población inicial-

In [12]:
CostoMin=min(Costos(:,1))
Cmin(1,1)=CostoMin;


CostoMin =

   1.7749e+05




Se genera las subsecuentes generaciones por medio de los operadores selección, cruce y mutación. Este proceso se repite un número GenMax de veces. 

In [13]:
% Repeticion de las generaciones
Generacion=1; % Para iniciar el while
while Generacion<GenMax
    % Creación de una nueva solución
    for i=1 : size(Vv,1)
        Solucion(1,i+1)=Poblacion(randi(Mu),i+1);
        
        if(rand()<=PM)
            [Mut]=EEMutacion(Solucion,Vv,i);
            Solucion(1,i+1)=Mut;
        else
            if(rand()<=1-PM)
                [Mut]=EEMutacion2(Vv,i);
                Solucion(1,i+1)=Mut;
            end
        end
    end
    % Se revisa la validez de la viga
    [Check,Le,B,H,fc,d,Asc,As1,As2,Av,sEApoyo,sECentro]=...
        ComprobacionesAG(Solucion,L,rec,BApoyo,Wm,w);
    Solucion(1,15)=Le;
    
    if(Check==0)
        [Costo]=CostoViga(L,Le,rec,BApoyo,B,H,fc,d,Asc,As1,As2,Av,sEApoyo,sECentro,Vfc);
        Generacion=Generacion+1;
        
        if(max(Costos)>Costo)
            j=find(max(Costos)==Costos,1);
            Poblacion(j,:)=Solucion;
            Costos(j,1)=Costo;
        end      
        CostoMin=min(Costos(:,1));
        Cmin(Generacion,1)=CostoMin;
        Solucion=zeros(1,15);
    end
end

## Resultados

Se guarda una de las vigas optimizadas encontradas y se imprime el costo de la solución de mayor calidad encontrada. 

In [14]:
VigaOptima=Poblacion(1,[2:15]);
CostoMin=min(Costos(:,1))
fprintf('Costo mínimo: %d MXN\n',CostoMin)


CostoMin =

   4.8020e+04

Costo mínimo: 4.801971e+04 MXN



Por último, se imprimen las características de la viga optimizada. 

In [15]:
fprintf('Datos de la sección:\n')
fprintf('Resistencia a compresión (fc): %d MPa\n',VigaOptima(1,1))
fprintf('Canto de la sección (H): %1.1f m\n',VigaOptima(1,2))
fprintf('Ancho de la sección (B): %1.1f m\n',VigaOptima(1,3))
fprintf('\n')

fprintf('Datos armado a compresión:\n')
fprintf('Número varillas a compresión (nc): %i varillas\n',VigaOptima(1,4))
fprintf('Diámetro varillas a compresión (fic): %2.1f in/8 \n',VigaOptima(1,5))
fprintf('\n')

fprintf('Datos armado a tensión superior:\n')
fprintf('Número varillas a tensión capa superior (ns1): %d varillas\n',VigaOptima(1,6))
fprintf('Diámetro varillas a tensión capa superior (fis1): %2.1f in/8 \n',VigaOptima(1,7))
fprintf('\n')

fprintf('Datos armado a tensión inferior:\n')
fprintf('Número varillas a tensión capa inferior (ns2): %d varillas\n',VigaOptima(1,8))
fprintf('Diámetro varillas a tensión capa inferior (fis2): %2.1f in/8 \n',VigaOptima(1,9))
fprintf('\n')

fprintf('Datos sobre estribos:\n')
fprintf('Número de estribos (nE): %d estribo\n',VigaOptima(1,10))
fprintf('Diámetro estribos (fiE): %2.1f in/8 \n',VigaOptima(1,11))
fprintf('Separación estribos centro (sE): %0.1f m \n',VigaOptima(1,13))

Datos de la sección:
Resistencia a compresión (fc): 70 MPa
Canto de la sección (H): 2.3 m
Ancho de la sección (B): 0.4 m

Datos armado a compresión:
Número varillas a compresión (nc): 2 varillas
Diámetro varillas a compresión (fic): 2.5 in/8 

Datos armado a tensión superior:
Número varillas a tensión capa superior (ns1): 2 varillas
Diámetro varillas a tensión capa superior (fis1): 8.0 in/8 

Datos armado a tensión inferior:
Número varillas a tensión capa inferior (ns2): 4 varillas
Diámetro varillas a tensión capa inferior (fis2): 12.0 in/8 

Datos sobre estribos:
Número de estribos (nE): 1 estribo
Diámetro estribos (fiE): 2.5 in/8 
Separación estribos centro (sE): 0.3 m 



## Referencias

[1] Coello, C. C., Hernández, F. S. & Farrera, F. A. Optimal design of reinforced concrete beams using genetic algorithms. Expert Syst. Appl. 12, 101–108 (1997).

[2] Gobierno de la Ciudad de México. Normas Técnicas Complementarias. (2017).

[3] Rechenberg, I. Evolutionsstrategie: Optimierung techischer Systeme nach Prinzipien der biologischen Evolution. Frommann Holzboog Verlag, Stuttgart. (1973).

[4] Bäck, T., Hoffmeister, F. & Schwefel, H. A survey of evolution strategies. (1991).